In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
from datetime import datetime

from sklearn import preprocessing

In [49]:
df = pd.read_csv('../data/korea_netflix.csv')
df2 = pd.read_csv('../data/crawling_movie_info_modified.csv')
df3 = pd.read_csv('../data/crawling_netizen_review_modified.csv')
df4 = pd.read_csv('../data/crawling_audience_review.csv')

In [50]:
movie_df = df2.copy()
netizen_df = df3.copy()
audience_df = df4.copy()

In [4]:
korea_netflix_df = df.copy()
korea_netflix_df = korea_netflix_df.rename(columns={'Unnamed: 0':'original index'})
# korea_netflix_df = korea_netflix_df.drop(df.columns[0], axis=1)
korea_netflix_df

,original index,Title,Release Date,Series or Movies,drama,comedy,action,thriller,romance,crime,...,music,short,sport,war,musical,reality-tv,western,game-show,talk-show,Title for Crawling
0,4,Moxie,2011-09-22,Movie,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,moxie
1,27,Comrades Almost a Love Story,1996-11-02,Movie,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,comradesalmostalovestory
2,28,The Closet,2001-08-10,Movie,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,thecloset
3,53,The Bold Type,2017-06-20,Series,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,theboldtype
4,55,Lobster Cop,2018-06-22,Movie,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,lobstercop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4840,15475,K-POP Extreme Survival,NaN,Series,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,kpopextremesurvival
4841,15476,DreamWorks Shreks Swamp Stories,NaN,Series,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,dreamworksshreksswampstories
4842,15477,DreamWorks Happy Holidays from Madagascar,NaN,Series,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,dreamworkshappyholidaysfrommadagascar
4843,15478,DreamWorks Holiday Classics,NaN,Series,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,dreamworksholidayclassics


In [5]:
korea_netflix_df.head()

,original index,Title,Release Date,Series or Movies,drama,comedy,action,thriller,romance,crime,...,music,short,sport,war,musical,reality-tv,western,game-show,talk-show,Title for Crawling
0,4,Moxie,2011-09-22,Movie,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,moxie
1,27,Comrades Almost a Love Story,1996-11-02,Movie,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,comradesalmostalovestory
2,28,The Closet,2001-08-10,Movie,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,thecloset
3,53,The Bold Type,2017-06-20,Series,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,theboldtype
4,55,Lobster Cop,2018-06-22,Movie,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,lobstercop


In [6]:
korea_netflix_df[['Title', 'Series or Movies', 'Title for Crawling']]

,Title,Series or Movies,Title for Crawling
0,Moxie,Movie,moxie
1,Comrades Almost a Love Story,Movie,comradesalmostalovestory
2,The Closet,Movie,thecloset
3,The Bold Type,Series,theboldtype
4,Lobster Cop,Movie,lobstercop
...,...,...,...
4840,K-POP Extreme Survival,Series,kpopextremesurvival
4841,DreamWorks Shreks Swamp Stories,Series,dreamworksshreksswampstories
4842,DreamWorks Happy Holidays from Madagascar,Series,dreamworkshappyholidaysfrommadagascar
4843,DreamWorks Holiday Classics,Series,dreamworksholidayclassics


## Title for Crawling List, Series or Movie == Movie

In [7]:
korea_netflix_df[korea_netflix_df['Series or Movies']=='Movie'].head()

,original index,Title,Release Date,Series or Movies,drama,comedy,action,thriller,romance,crime,...,music,short,sport,war,musical,reality-tv,western,game-show,talk-show,Title for Crawling
0,4,Moxie,2011-09-22,Movie,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,moxie
1,27,Comrades Almost a Love Story,1996-11-02,Movie,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,comradesalmostalovestory
2,28,The Closet,2001-08-10,Movie,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,thecloset
4,55,Lobster Cop,2018-06-22,Movie,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,lobstercop
5,56,Magic hour,NaN,Movie,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,magichour


- 영화 The Closet이 comedy??

In [8]:
korea_netflix_df.iloc[2,:]
# 음...

original index                28
Title                 The Closet
Release Date          2001-08-10
Series or Movies           Movie
drama                          0
comedy                         1
action                         0
thriller                       0
romance                        0
crime                          0
adventure                      0
animation                      0
fantasy                        0
family                         0
sci-fi                         0
mystery                        0
horror                         0
documentary                    0
biography                      0
history                        0
music                          0
short                          0
sport                          0
war                            0
musical                        0
reality-tv                     0
western                        0
game-show                      0
talk-show                      0
Title for Crawling     thecloset
Name: 2, d

###  title_for_crawling_list 
- title_for_crawling_list << 이 리스트로 크롤링 하면 될 듯 하네요

In [9]:
title_for_crawling_list = korea_netflix_df[korea_netflix_df['Series or Movies']=='Movie']['Title for Crawling'].values.tolist()

In [10]:
print(len(title_for_crawling_list), korea_netflix_df[korea_netflix_df['Series or Movies'] == 'Movie']['Title'].count())

3075 3075


In [11]:
korea_netflix_df.head()

,original index,Title,Release Date,Series or Movies,drama,comedy,action,thriller,romance,crime,...,music,short,sport,war,musical,reality-tv,western,game-show,talk-show,Title for Crawling
0,4,Moxie,2011-09-22,Movie,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,moxie
1,27,Comrades Almost a Love Story,1996-11-02,Movie,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,comradesalmostalovestory
2,28,The Closet,2001-08-10,Movie,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,thecloset
3,53,The Bold Type,2017-06-20,Series,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,theboldtype
4,55,Lobster Cop,2018-06-22,Movie,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,lobstercop


# Crawled Movie Info

In [12]:
movie_df

,code,korean_title,english_title,genre_list,country,released_date,director,actors
0,203096,걸스 오브 막시,moxie,['드라마'],['미국'],NaN,에이미 포엘러,"['조세핀 랭포드', ' 패트릭 슈왈제네거더보기']"
1,17997,첨밀밀,comradesalmostalovestory,"['멜로/로맨스', '드라마']",['홍콩'],1997.03.01,진가신,"['여명(소군)', ' 장만옥(이요)더보기']"
2,181925,클로젯,thecloset,"['미스터리', '드라마']",['한국'],2020.02.05,김광빈,"['하정우(상원)', ' 김남길(경훈)', ' 허율(이나)더보기']"
3,195979,비밀경찰: 랍스터 캅,lobstercop,"['액션', '코미디']",['중국'],NaN,이흔운,"['왕첸웬', ' 원산산', ' 류화더보기']"
4,138113,매직 아워,magichour,['범죄'],['미국'],NaN,콜 뮐러,"['트레버 모건(레이)', ' 크리스 렘세(딜런 폭스)', ' 윌리엄 러스(브루스 ..."
...,...,...,...,...,...,...,...,...
1819,72043,쓰리 데이즈,thenextthreedays,"['범죄', '드라마', '멜로/로맨스', '스릴러']",['미국'],2010.12.22,폴 해기스,"['러셀 크로우(존 브레넌)', ' 엘리자베스 뱅크스(라라 브레넌)더보기']"
1820,77566,브레이킹 던 part2,thetwilightsagabreakingdawnpart2,"['드라마', '판타지', '멜로/로맨스', '스릴러']",['미국'],2012.11.15,빌 콘돈,"['크리스틴 스튜어트(벨라 컬렌)', ' 로버트 패틴슨(에드워드 컬렌)', ' 테일..."
1821,38227,첫 키스만 50번째,50firstdates,"['멜로/로맨스', '코미디']",['미국'],2017.06.22 2004.04.15,피터 시걸,"['아담 샌들러(헨리 로스)', ' 드류 베리모어(루시 윗모어)더보기']"
1822,74567,브레이킹 던 part1,thetwilightsagabreakingdawnpart1,"['드라마', '판타지', '멜로/로맨스', '스릴러']",['미국'],2011.11.30,빌 콘돈,"['크리스틴 스튜어트(벨라 스완)', ' 로버트 패틴슨(에드워드 컬렌)더보기']"


In [13]:
print(movie_df['genre_list'][0], type(movie_df['genre_list'][0]))
movie_df['genre_list'][0]

['드라마'] <class 'str'>


"['드라마']"

## Dealing with genre_list

In [14]:
# https://towardsdatascience.com/dealing-with-list-values-in-pandas-dataframes-a177e534f173
for i, l in enumerate(movie_df['genre_list'][:10]):
    print('list',i,'is',type(l))

list 0 is <class 'str'>
list 1 is <class 'str'>
list 2 is <class 'str'>
list 3 is <class 'str'>
list 4 is <class 'str'>
list 5 is <class 'str'>
list 6 is <class 'str'>
list 7 is <class 'str'>
list 8 is <class 'str'>
list 9 is <class 'str'>


In [15]:
movie_df['genre_list'] = movie_df['genre_list'].apply(eval)

In [16]:
for i, l in enumerate(movie_df['genre_list'][:10]):
    print('list',i,'is',type(l))

list 0 is <class 'list'>
list 1 is <class 'list'>
list 2 is <class 'list'>
list 3 is <class 'list'>
list 4 is <class 'list'>
list 5 is <class 'list'>
list 6 is <class 'list'>
list 7 is <class 'list'>
list 8 is <class 'list'>
list 9 is <class 'list'>


In [17]:
print(movie_df['genre_list'][1], type(movie_df['genre_list'][1]))
movie_df['genre_list'][1][0]

['멜로/로맨스', '드라마'] <class 'list'>


'멜로/로맨스'

##  Unique Value from Genre_List

In [18]:
genre_list = []
for i in range(len(movie_df['genre_list'])):
    try:
        for j in range(len(movie_df['genre_list'][i])):
            if movie_df['genre_list'][i][j] not in genre_list:
                genre_list.append(movie_df['genre_list'][i][j].strip())
    except:
        continue

In [19]:
unique_genre_list = list(set(genre_list))

print(len(unique_genre_list))
unique_genre_list

23


['판타지',
 '멜로/로맨스',
 '컬트',
 '애니메이션',
 '뮤지컬',
 '가족',
 '공연실황',
 '전쟁',
 '스릴러',
 '느와르',
 '무협',
 '미스터리',
 '드라마',
 '모험',
 '코미디',
 '액션',
 '서스펜스',
 '공포',
 '다큐멘터리',
 'SF',
 '서부',
 '범죄',
 '블랙코미디']

### function to decompose genre_list

In [20]:
def to_1D(series):
 return pd.Series([x for _list in series for x in _list])

In [21]:
to_1D(movie_df['genre_list']).value_counts()

드라마       806
코미디       514
액션        425
스릴러       331
멜로/로맨스    267
범죄        190
모험        185
애니메이션     133
판타지       124
공포        122
SF        114
미스터리       99
가족         86
다큐멘터리      74
전쟁         39
뮤지컬        20
느와르        10
서부          8
서스펜스        7
무협          1
컬트          1
블랙코미디       1
공연실황        1
dtype: int64

### Create a dict from genre_list

In [22]:
values = to_1D(movie_df['genre_list']).value_counts().keys().tolist()
counts = to_1D(movie_df['genre_list']).value_counts().tolist()
unique_items = dict(zip(values, counts))
unique_items

{'드라마': 806,
 '코미디': 514,
 '액션': 425,
 '스릴러': 331,
 '멜로/로맨스': 267,
 '범죄': 190,
 '모험': 185,
 '애니메이션': 133,
 '판타지': 124,
 '공포': 122,
 'SF': 114,
 '미스터리': 99,
 '가족': 86,
 '다큐멘터리': 74,
 '전쟁': 39,
 '뮤지컬': 20,
 '느와르': 10,
 '서부': 8,
 '서스펜스': 7,
 '무협': 1,
 '컬트': 1,
 '블랙코미디': 1,
 '공연실황': 1}

###  One Hot Encoding

- https://towardsdatascience.com/dealing-with-list-values-in-pandas-dataframes-a177e534f173

In [23]:
def boolean_df(item_lists, unique_items):
# Create empty dict
    bool_dict = {}
    
    # Loop through all the tags
    for i, item in enumerate(unique_items):
        
        # Apply boolean mask
        bool_dict[item] = item_lists.apply(lambda x: item in x)
            
    # Return the results as a dataframe
    return pd.DataFrame(bool_dict)

In [24]:
genre_bool_df = boolean_df(movie_df['genre_list'], unique_items)

In [25]:
genre_bool_df.head()

,드라마,코미디,액션,스릴러,멜로/로맨스,범죄,모험,애니메이션,판타지,공포,...,다큐멘터리,전쟁,뮤지컬,느와르,서부,서스펜스,무협,컬트,블랙코미디,공연실황
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [26]:
genre_int_df = genre_bool_df.astype(int)

In [27]:
to_rename = {'드라마': 'drama',
 '코미디': 'comedy',
 '액션': 'action',
 '스릴러': 'thriller',
 '멜로/로맨스': 'romance',
 '범죄': 'crime',
 '모험': 'adventure',
 '애니메이션': 'animation',
 '판타지': 'fantasy',
 '공포': 'horror',
 'SF': 'sci-fi',
 '미스터리': 'mystery',
 '가족': 'family',
 '다큐멘터리': 'documentary',
 '전쟁': 'war',
 '뮤지컬': 'musical',
 '서부': 'western'}

In [28]:
genre_int_df.head()

,드라마,코미디,액션,스릴러,멜로/로맨스,범죄,모험,애니메이션,판타지,공포,...,다큐멘터리,전쟁,뮤지컬,느와르,서부,서스펜스,무협,컬트,블랙코미디,공연실황
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
{'드라마': 806,
 '코미디': 514,
 '액션': 425,
 '스릴러': 331,
 '멜로/로맨스': 267,
 '범죄': 190,
 '모험': 185,
 '애니메이션': 133,
 '판타지': 124,
 '공포': 122,
 'SF': 114,
 '미스터리': 99,
 '가족': 86,
 '다큐멘터리': 74,
 '전쟁': 39,
 '뮤지컬': 20,
 '느와르': 10,
 '서부': 8,
 '서스펜스': 7,
 '무협': 1,
 '블랙코미디': 1,
 '컬트': 1,
 '공연실황': 1}

{'드라마': 806,
 '코미디': 514,
 '액션': 425,
 '스릴러': 331,
 '멜로/로맨스': 267,
 '범죄': 190,
 '모험': 185,
 '애니메이션': 133,
 '판타지': 124,
 '공포': 122,
 'SF': 114,
 '미스터리': 99,
 '가족': 86,
 '다큐멘터리': 74,
 '전쟁': 39,
 '뮤지컬': 20,
 '느와르': 10,
 '서부': 8,
 '서스펜스': 7,
 '무협': 1,
 '블랙코미디': 1,
 '컬트': 1,
 '공연실황': 1}

In [30]:
movie_ohe_df = genre_int_df.drop(columns=['느와르', '서스펜스', '무협', '블랙코미디', '컬트', '공연실황'])

In [31]:
movie_ohe_df = movie_ohe_df.rename(columns = to_rename)

In [32]:
movie_ohe_w_t_df = movie_ohe_df.copy()

In [33]:
movie_ohe_w_t_df = pd.concat([movie_df.iloc[:, 0:2], movie_ohe_df], axis=1)

In [35]:
movie_ohe_w_t_df.head()

,code,korean_title,drama,comedy,action,thriller,romance,crime,adventure,animation,fantasy,horror,sci-fi,mystery,family,documentary,war,musical,western
0,203096,걸스 오브 막시,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,17997,첨밀밀,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,181925,클로젯,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,195979,비밀경찰: 랍스터 캅,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,138113,매직 아워,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [89]:
movie_ohe_w_t_df.columns

Index(['code', 'korean_title', 'drama', 'comedy', 'action', 'thriller',
       'romance', 'crime', 'adventure', 'animation', 'fantasy', 'horror',
       'sci-fi', 'mystery', 'family', 'documentary', 'war', 'musical',
       'western'],
      dtype='object')

In [41]:
movie_ohe_w_t_df['code'].count()

1824

# Crawled Netizen Info 

In [45]:
netizen_df.head()

,code,korean_title,num_of_netizen,key_review,score_male,score_female,scores_per_age
0,203096,걸스 오브 막시,140,이 영화는 20대 여성이 좋아하는 스토리가 뛰어난 영화입니다.,6.12,9.86,"[9.76, 9.83, 8.92, 9.38, 10.0]"
1,17997,첨밀밀,"1,851",이 영화는 20대 남성이 좋아하는 연기가 뛰어난 영화입니다.,9.19,9.13,"[8.86, 9.05, 9.27, 9.48, 9.35]"
2,181925,클로젯,"6,242",이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,5.55,7.04,"[7.66, 6.11, 5.95, 6.11, 6.33]"
3,197517,마리오네트,121,이 영화는 40대 여성이 좋아하는 스토리가 뛰어난 영화입니다.,7.42,8.25,"[10.0, 7.89, 8.28, 7.52, 6.6]"
4,155356,미쓰백,"8,671",이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,8.29,9.60,"[9.69, 9.36, 8.7, 8.55, 8.51]"


In [42]:
netizen_df.count()

code              1207
korean_title      1207
num_of_netizen    1207
key_review        1174
score_male        1207
score_female      1207
scores_per_age    1207
dtype: int64

#  Crawled Audience Info

In [51]:
audience_df.count()

code               1824
korean_title       1824
num_of_audience     594
key_review          594
score_male          594
score_female        594
scores_per_age      594
num_per_genders     426
dtype: int64

In [52]:
audience_df.head()

,code,korean_title,num_of_audience,key_review,score_male,score_female,scores_per_age,num_per_genders
0,203096,걸스 오브 막시,NaN,NaN,NaN,NaN,NaN,NaN
1,17997,첨밀밀,NaN,NaN,NaN,NaN,NaN,NaN
2,181925,클로젯,387,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,7.93,8.41,"[8.75, 7.75, 8.17, 8.79, 8.63]","['58%', '42%']"
3,195979,비밀경찰: 랍스터 캅,NaN,NaN,NaN,NaN,NaN,NaN
4,138113,매직 아워,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
audience_df[(~audience_df['scores_per_age'].isna()) & (audience_df['num_per_genders'].isna())]

,code,korean_title,num_of_audience,key_review,score_male,score_female,scores_per_age,num_per_genders
309,130267,다시..올래,2,이 영화는 30대 여성이 좋아하는 영화입니다.,10.00,0.00,"[0.0, 0.0, 10.0, 10.0, 0.0]",NaN
463,159146,부에나 비스타 소셜 클럽2: 아디오스,24,이 영화는 30대 남성이 좋아하는 영화입니다.,9.00,8.80,"[0.0, 9.33, 9.18, 10.0, 7.2]",NaN
733,145376,최강전사 미니특공대 : 영웅의 탄생,495,이 영화는 30대 여성이 좋아하는 영화입니다.,8.11,8.58,"[8.0, 8.69, 8.52, 8.18, 5.0]",NaN
757,160720,아기돼지 3형제와 매직램프,102,이 영화는 30대 여성이 좋아하는 영화입니다.,7.94,7.44,"[0.0, 8.33, 7.46, 7.6, 7.0]",NaN
760,156930,파리 투 마르세유: 2주간의 여행,2,이 영화는 20대 여성이 좋아하는 영화입니다.,8.00,9.00,"[0.0, 0.0, 8.0, 0.0, 9.0]",NaN
...,...,...,...,...,...,...,...,...
1799,87556,올드보이,16,이 영화는 30대 남성이 좋아하는 연기가 뛰어난 영화입니다.,6.82,5.00,"[0.0, 7.0, 5.67, 0.0, 0.0]",NaN
1806,86843,케빈에 대하여,7,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,8.00,9.60,"[0.0, 9.2, 9.0, 0.0, 0.0]",NaN
1807,52553,향수 - 어느 살인자의 이야기,66,이 영화는 20대 남성이 좋아하는 연출이 뛰어난 영화입니다.,8.93,9.05,"[9.33, 9.16, 8.79, 8.33, 7.5]",NaN
1809,34225,워크 투 리멤버,3,이 영화는 10대 여성이 좋아하는 연기가 뛰어난 영화입니다.,9.00,10.00,"[10.0, 9.5, 0.0, 0.0, 0.0]",NaN


### Code list - empty num_per_genders

In [90]:
audience_gender_code_list = audience_df[(~audience_df['scores_per_age'].isna()) & (audience_df['num_per_genders'].isna())]['code'].tolist()
audience_gender_code_list

[130267,
 159146,
 145376,
 160720,
 156930,
 159785,
 72058,
 35901,
 109905,
 74027,
 181030,
 144271,
 129012,
 103331,
 124212,
 67769,
 130041,
 136007,
 118917,
 109955,
 193839,
 104478,
 127865,
 130983,
 129331,
 129049,
 122469,
 192121,
 106557,
 129333,
 95541,
 172819,
 117802,
 122596,
 118980,
 90596,
 15899,
 127335,
 69956,
 107352,
 118950,
 19303,
 18781,
 103708,
 32686,
 101961,
 94187,
 73318,
 112039,
 101236,
 117790,
 105521,
 88426,
 122535,
 120123,
 93739,
 108225,
 112079,
 93028,
 99096,
 61101,
 109778,
 87309,
 114249,
 190727,
 122473,
 127321,
 146212,
 109982,
 195970,
 125841,
 45290,
 118909,
 72524,
 178654,
 164115,
 113377,
 109095,
 16792,
 149248,
 109169,
 100676,
 94829,
 137908,
 113348,
 118397,
 99740,
 191929,
 92075,
 52555,
 90589,
 66158,
 118367,
 12787,
 125438,
 137327,
 155411,
 160749,
 162824,
 125494,
 97631,
 72054,
 18878,
 91192,
 103765,
 97858,
 97612,
 189150,
 100691,
 103324,
 95203,
 78726,
 124290,
 86335,
 32667,
 105

# Merge Netizen & Audience by Code for key_review

In [55]:
net_aud_key_df = pd.merge(netizen_df[['code', 'key_review']], audience_df[['code', 'key_review']], how='outer', on='code')
net_aud_key_df

,code,key_review_x,key_review_y
0,203096,이 영화는 20대 여성이 좋아하는 스토리가 뛰어난 영화입니다.,NaN
1,17997,이 영화는 20대 남성이 좋아하는 연기가 뛰어난 영화입니다.,NaN
2,181925,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.
3,197517,이 영화는 40대 여성이 좋아하는 스토리가 뛰어난 영화입니다.,이 영화는 40대 여성이 좋아하는 스토리가 뛰어난 영화입니다.
4,155356,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.
...,...,...,...
1819,40211,NaN,NaN
1820,154212,NaN,NaN
1821,72062,NaN,NaN
1822,43574,NaN,NaN


In [75]:
key_review_word_list = ['남성', '여성', '연출', '연기', '스토리', '영상미', 'OST', '10대', '20대', '30대', '40대', '50대']

In [57]:
# netizen audience not none 
net_aud_key_nn_df = net_aud_key_df[(~net_aud_key_df['key_review_x'].isna()) & (~net_aud_key_df['key_review_y'].isna())]
net_aud_key_nn_df

,code,key_review_x,key_review_y
2,181925,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.
3,197517,이 영화는 40대 여성이 좋아하는 스토리가 뛰어난 영화입니다.,이 영화는 40대 여성이 좋아하는 스토리가 뛰어난 영화입니다.
4,155356,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.
5,168668,이 영화는 40대 남성이 좋아하는 스토리가 뛰어난 영화입니다.,이 영화는 40대 남성이 좋아하는 스토리가 뛰어난 영화입니다.
7,143776,"이 영화는 40대 남성이 좋아하는 영상미, 스토리가 뛰어난 영화입니다.","이 영화는 40대 남성이 좋아하는 영상미, 스토리가 뛰어난 영화입니다."
...,...,...,...
1188,87556,이 영화는 30대 남성이 좋아하는 연기가 뛰어난 영화입니다.,이 영화는 30대 남성이 좋아하는 연기가 뛰어난 영화입니다.
1192,86843,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.
1193,52553,이 영화는 20대 남성이 좋아하는 연출이 뛰어난 영화입니다.,이 영화는 20대 남성이 좋아하는 연출이 뛰어난 영화입니다.
1195,34225,이 영화는 10대 여성이 좋아하는 연기가 뛰어난 영화입니다.,이 영화는 10대 여성이 좋아하는 연기가 뛰어난 영화입니다.


In [60]:
target_review = net_aud_key_nn_df['key_review_x'][2]
target_review

'이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.'

In [61]:
type(target_review)

str

###  pick key words from key_review

In [81]:
net_aud_key_nn_df['key_netizen'] = net_aud_key_nn_df['key_review_x'].apply(lambda x: [s for s in key_review_word_list if s in x])


<ipython-input-81-687f7aea8e59>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  net_aud_key_nn_df['key_netizen'] = net_aud_key_nn_df['key_review_x'].apply(lambda x: [s for s in key_review_word_list if s in x])


In [83]:
net_aud_key_nn_df['key_audience'] = net_aud_key_nn_df['key_review_y'].apply(lambda x: [s for s in key_review_word_list if s in x])

<ipython-input-83-073c3f7abddd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  net_aud_key_nn_df['key_audience'] = net_aud_key_nn_df['key_review_y'].apply(lambda x: [s for s in key_review_word_list if s in x])


In [84]:
net_aud_key_nn_df

,code,key_review_x,key_review_y,key_netizen,key_audience
2,181925,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,"[여성, 연기, 20대]","[여성, 연기, 20대]"
3,197517,이 영화는 40대 여성이 좋아하는 스토리가 뛰어난 영화입니다.,이 영화는 40대 여성이 좋아하는 스토리가 뛰어난 영화입니다.,"[여성, 스토리, 40대]","[여성, 스토리, 40대]"
4,155356,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,"[여성, 연기, 20대]","[여성, 연기, 20대]"
5,168668,이 영화는 40대 남성이 좋아하는 스토리가 뛰어난 영화입니다.,이 영화는 40대 남성이 좋아하는 스토리가 뛰어난 영화입니다.,"[남성, 스토리, 40대]","[남성, 스토리, 40대]"
7,143776,"이 영화는 40대 남성이 좋아하는 영상미, 스토리가 뛰어난 영화입니다.","이 영화는 40대 남성이 좋아하는 영상미, 스토리가 뛰어난 영화입니다.","[남성, 스토리, 영상미, 40대]","[남성, 스토리, 영상미, 40대]"
...,...,...,...,...,...
1188,87556,이 영화는 30대 남성이 좋아하는 연기가 뛰어난 영화입니다.,이 영화는 30대 남성이 좋아하는 연기가 뛰어난 영화입니다.,"[남성, 연기, 30대]","[남성, 연기, 30대]"
1192,86843,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,"[여성, 연기, 20대]","[여성, 연기, 20대]"
1193,52553,이 영화는 20대 남성이 좋아하는 연출이 뛰어난 영화입니다.,이 영화는 20대 남성이 좋아하는 연출이 뛰어난 영화입니다.,"[남성, 연출, 20대]","[남성, 연출, 20대]"
1195,34225,이 영화는 10대 여성이 좋아하는 연기가 뛰어난 영화입니다.,이 영화는 10대 여성이 좋아하는 연기가 뛰어난 영화입니다.,"[여성, 연기, 10대]","[여성, 연기, 10대]"


- both key words from audience's & netizen's key_reviews are the same

In [85]:
net_aud_key_nn_df[net_aud_key_nn_df['key_netizen'] == net_aud_key_nn_df['key_audience']]

,code,key_review_x,key_review_y,key_netizen,key_audience
2,181925,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,"[여성, 연기, 20대]","[여성, 연기, 20대]"
3,197517,이 영화는 40대 여성이 좋아하는 스토리가 뛰어난 영화입니다.,이 영화는 40대 여성이 좋아하는 스토리가 뛰어난 영화입니다.,"[여성, 스토리, 40대]","[여성, 스토리, 40대]"
4,155356,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,"[여성, 연기, 20대]","[여성, 연기, 20대]"
5,168668,이 영화는 40대 남성이 좋아하는 스토리가 뛰어난 영화입니다.,이 영화는 40대 남성이 좋아하는 스토리가 뛰어난 영화입니다.,"[남성, 스토리, 40대]","[남성, 스토리, 40대]"
7,143776,"이 영화는 40대 남성이 좋아하는 영상미, 스토리가 뛰어난 영화입니다.","이 영화는 40대 남성이 좋아하는 영상미, 스토리가 뛰어난 영화입니다.","[남성, 스토리, 영상미, 40대]","[남성, 스토리, 영상미, 40대]"
...,...,...,...,...,...
1188,87556,이 영화는 30대 남성이 좋아하는 연기가 뛰어난 영화입니다.,이 영화는 30대 남성이 좋아하는 연기가 뛰어난 영화입니다.,"[남성, 연기, 30대]","[남성, 연기, 30대]"
1192,86843,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,"[여성, 연기, 20대]","[여성, 연기, 20대]"
1193,52553,이 영화는 20대 남성이 좋아하는 연출이 뛰어난 영화입니다.,이 영화는 20대 남성이 좋아하는 연출이 뛰어난 영화입니다.,"[남성, 연출, 20대]","[남성, 연출, 20대]"
1195,34225,이 영화는 10대 여성이 좋아하는 연기가 뛰어난 영화입니다.,이 영화는 10대 여성이 좋아하는 연기가 뛰어난 영화입니다.,"[여성, 연기, 10대]","[여성, 연기, 10대]"


#####  list of movie which has netizen's key_review but no audience's 

In [87]:
net_aud_key_df[(~net_aud_key_df['key_review_x'].isna()) & (net_aud_key_df['key_review_y'].isna())]

,code,key_review_x,key_review_y
0,203096,이 영화는 20대 여성이 좋아하는 스토리가 뛰어난 영화입니다.,NaN
1,17997,이 영화는 20대 남성이 좋아하는 연기가 뛰어난 영화입니다.,NaN
6,151155,"이 영화는 30대 남성이 좋아하는 스토리, 연출이 뛰어난 영화입니다.",NaN
24,171744,이 영화는 30대 남성이 좋아하는 연출이 뛰어난 영화입니다.,NaN
39,194398,이 영화는 20대 여성이 좋아하는 영상미가 뛰어난 영화입니다.,NaN
...,...,...,...
1201,36755,이 영화는 10대 남성이 좋아하는 연기가 뛰어난 영화입니다.,NaN
1202,72043,이 영화는 20대 남성이 좋아하는 스토리가 뛰어난 영화입니다.,NaN
1203,77566,이 영화는 10대 여성이 좋아하는 스토리가 뛰어난 영화입니다.,NaN
1205,74567,이 영화는 10대 여성이 좋아하는 연기가 뛰어난 영화입니다.,NaN
